<img src="images/img.png" />

# CS5228 Project, Group 32

In [1]:
# Auto reload
%load_ext autoreload
%autoreload 2

## Data Preprocessing
In this part, we are going to perform some data preprocessing steps. This may include:
* Data cleaning: handle missing values, duplicates, inconsistant or invalid vallues, outliers

* Data reduction: reduce number of attributes, reduce number of attribute values

* Data transformation: attribute construction, normalization

* Data discretization: encode to numerical attributes

### Setting up the Notebook

In [2]:
import os
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Load file into pandas dataframe
df = pd.read_csv('./data/train.csv')

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points in training data, each with 30 attributes.


### Data Cleaning

Before data cleaning, remove the known attributes that are not meaningful to our prediction model:
  * Meaningless idendifier: listing_id 
  * Attributes in free text: title, description, features, accessories
  * Attribute with the same value: eco_category, indicative_price
  * Attribute unlikely to affect price: curb_weight

In [4]:
columns_to_drop = [
    'listing_id',          # Meaningless identifier
    'title',               # Attributes in free text
    'description',
    'original_reg_date',
    # 'curb_weight',         # Attribute unlikely to affect price
    'lifespan',
    'eco_category',        # Attribute with the same value
    'features',
    'accessories',
    'indicative_price',
]

df = df.drop(columns=columns_to_drop)

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points in training data, each with 21 attributes.


### Handle Missing Values
Firstly, for each of the columns with missing value, check the number of rows with NaN values.
There are 3 scenarios:
1. NaN value is the major (e.g. fuel_type has 19121 rows with NaN values), we remove the corresponding attritubes.
2. NaN value is the minor. We can choose to fill or delete related data points. 

In [5]:
columns_to_check = [
    'make',
    'fuel_type',
    'manufactured',
    'power',
    'engine_cap',
    'mileage',
    'no_of_owners',
    'depreciation',
    'road_tax',
    'dereg_value',
    'omv',
    'arf',
    'opc_scheme'
]

# Calculate the number of NaN values in each specified column
nan_counts = df[columns_to_check].isna().sum()

# Print the number of NaN values for each column
print('Training data')
for column, count in nan_counts.items():
    print(f"Column '{column}' has {count} rows with NaN values.")

Training data
Column 'make' has 1316 rows with NaN values.
Column 'fuel_type' has 19121 rows with NaN values.
Column 'manufactured' has 7 rows with NaN values.
Column 'power' has 2640 rows with NaN values.
Column 'engine_cap' has 596 rows with NaN values.
Column 'mileage' has 5304 rows with NaN values.
Column 'no_of_owners' has 18 rows with NaN values.
Column 'depreciation' has 507 rows with NaN values.
Column 'road_tax' has 2632 rows with NaN values.
Column 'dereg_value' has 220 rows with NaN values.
Column 'omv' has 64 rows with NaN values.
Column 'arf' has 174 rows with NaN values.
Column 'opc_scheme' has 24838 rows with NaN values.


We delete attributes with TOO many NaN value here.

In [6]:
columns_to_drop_nan = [
    'fuel_type',
    'opc_scheme'
]

df = df.drop(columns=columns_to_drop_nan)

Then we try to fill up other missing values.

### Transform date time attributes to numerical values

In [7]:
df['reg_date'] = pd.to_datetime(df['reg_date'], format='%d-%b-%Y')
df['reg_year'] = df['reg_date'].dt.year
df = df.drop(columns=['reg_date'])

num_records, num_attributes = df.shape
print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))
# df.to_csv("./data/processing.csv", index=False)

There are 25000 data points, each with 19 attributes.


### Handling missing values in train

In [8]:
from util.DataPreprocess import HandlingMissingValues

df = HandlingMissingValues(df)


NaN values after handling:  0


D:\PythonWorkspace\CS5228_Project\util\DataPreprocess.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'mileage'] = df['mileage'].fillna(mean_values)
D:\PythonWorkspace\CS5228_Project\util\DataPreprocess.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'mileage'] = df['mileage'].round()
D:\PythonWorkspace\CS5228_Project\util\DataPreprocess.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

### Remove Exact Duplicates
We remove duplicated data points here.

In [9]:
df = df.drop_duplicates()

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))


There are 24258 data points in training data, each with 19 attributes.


### Merge rows with fewer data points on specific attributes(discarded idea)

### Transform categorical value to numerical values

In [10]:
categorical_columns = [
    'make',
    'model',
    'type_of_vehicle',
    'transmission',
]

encode_dict = {}
le = LabelEncoder()
for column in categorical_columns:
    df[column] = le.fit_transform(df[column])
    encode_dict[column] = {str(label): int(index) for index, label in enumerate(le.classes_)}

with open('./data/encode.json', 'w') as file:
    json.dump(encode_dict, file, indent=4)

### Handle category attribute

In [11]:
from util.DataPreprocess import HandlingCategoryAttribute

df = HandlingCategoryAttribute(df)

Number of unique categories: 15
Unique categories: {'low mileage car', 'consignment car', 'rare & exotic', 'hybrid cars', 'opc car', 'imported used vehicle', 'vintage cars', 'parf car', 'premium ad car', 'almost new car', 'direct owner sale', 'sta evaluated car', 'coe car', 'sgcarmart warranty cars', 'electric cars'}
There are 24258 data points, each with 33 attributes.


### Remove outliers

In [12]:
# from util.DataPreprocess import OutlierRemoval

# df = OutlierRemoval(df, 'model', 'price')

### Saving the Data

In [13]:
file_name = './data/train_preprocessed.csv'

# Check if the file exists
if os.path.exists(file_name):
    # Delete the file
    os.remove(file_name)
    print(f"Existing file '{file_name}' has been deleted.")

# Save the DataFrame to CSV
df.to_csv(file_name, index=False)
print(f"DataFrame has been saved to '{file_name}'.")

Existing file './data/train_preprocessed.csv' has been deleted.
DataFrame has been saved to './data/train_preprocessed.csv'.


## Data Mining

### Load preprocessed training data

In [14]:
# Load file into pandas dataframe, we saved our preprocessed file at path 'output_file'
training_file = './data/train_preprocessed.csv'
df = pd.read_csv(training_file)

columns_to_keep = [
    'model',
    'mileage',
    'low mileage car',
    'manufactured',
    'reg_year',
    'type_of_vehicle',
    'dereg_value',
    'depreciation',
    'power',
    'coe',
    'arf',
    'omv',
    'price',
    'engine_cap',
    'road_tax',
    'almost new car',
    'coe car',
    'parf car',
]

df = df[columns_to_keep]
columns_to_keep = [col for col in df.columns if col != 'price']

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))

There are 24258 data points in training data, each with 18 attributes.


### Data Augmentation, copy rows with less than 5 samples by group

In [15]:
from util.DataPreprocess import DataAugmentation

df_aug = DataAugmentation(df)

num_records, num_attributes = df_aug.shape
print("There are {} data points after augmentation, each with {} attributes.". format(num_records, num_attributes))

There are 40290 data points after augmentation, each with 18 attributes.


### Train_preprocessed.csv

In [16]:
file_name = './data/train_preprocessed.csv'

# Check if the file exists
if os.path.exists(file_name):
    # Delete the file
    os.remove(file_name)
    print(f"Existing file '{file_name}' has been deleted.")

# Save the DataFrame to CSV
df_aug.to_csv(file_name, index=False)
print(f"DataFrame has been saved to '{file_name}'.")

Existing file './data/train_preprocessed.csv' has been deleted.
DataFrame has been saved to './data/train_preprocessed.csv'.


### Load test data and preprocess reg_date and category

In [17]:
test_file = './data/test.csv'
df_test = pd.read_csv(test_file)

df_test['reg_date'] = pd.to_datetime(df_test['reg_date'], format='%d-%b-%Y')
df_test['reg_year'] = df_test['reg_date'].dt.year
df_test = df_test.drop(columns=['reg_date'])

# Replace '-' with an empty string
df_test['category'] = df_test['category'].replace('-', '')

# Split the 'category' column into lists
df_test['category_list'] = df_test['category'].str.split(', ')

# Handle empty strings by replacing them with empty lists
df_test['category_list'] = df_test['category_list'].apply(lambda x: [] if x == [''] else x)

# Import itertools for flattening lists
from itertools import chain

# Flatten the list of lists to a single list
all_categories = list(chain.from_iterable(df_test['category_list']))

# Get the unique categories
unique_categories = set(all_categories)

# Print the number of unique categories
print(f"Number of unique categories: {len(unique_categories)}")
print("Unique categories:", unique_categories)

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the category lists
category_dummies = mlb.fit_transform(df_test['category_list'])

# Create a DataFrame with the one-hot encoded categories
category_df = pd.DataFrame(category_dummies, columns=mlb.classes_, index=df_test.index)

# Concatenate the new dummy columns to the original DataFrame
df_test = pd.concat([df_test, category_df], axis=1)

# Drop the temporary 'category_list' column if desired
df_test.drop('category_list', axis=1, inplace=True)
df_test.drop('category', axis=1, inplace=True)

num_records, num_attributes = df_test.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

Number of unique categories: 15
Unique categories: {'low mileage car', 'consignment car', 'rare & exotic', 'hybrid cars', 'opc car', 'imported used vehicle', 'vintage cars', 'parf car', 'premium ad car', 'almost new car', 'direct owner sale', 'sta evaluated car', 'coe car', 'sgcarmart warranty cars', 'electric cars'}
There are 10000 data points, each with 43 attributes.


### Select attributes on test data to keep the same attribute with train_preprocessed

In [18]:
num_records, num_attributes = df_test.shape
print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

categorical_columns = [
    'make',
    'model',
    'type_of_vehicle',
    'transmission',
]

with open('./data/encode.json', 'r') as file:
    data = json.load(file)

for col, cate_dict in data.items():
    if col in df_test.columns:
        df_test[col] = df_test[col].map(cate_dict)

df_test = df_test[columns_to_keep]

num_records, num_attributes = df_test.shape
print("There are {} data points in test data, each with {} attributes.". format(num_records, num_attributes))
# df_test.to_csv('./data/test_preprocessed.csv', index=False)

There are 10000 data points, each with 43 attributes.
There are 10000 data points in test data, each with 17 attributes.


### Before handling nan values in test_file

In [19]:
file_name = './data/test_preprocessed.csv'

# Check if the file exists
if os.path.exists(file_name):
    # Delete the file
    os.remove(file_name)
    print(f"Existing file '{file_name}' has been deleted.")

# Save the Test est DataFrame to CSV
df_test.to_csv(file_name, index=False)
print(f"Test dataFrame has been saved to '{file_name}'.")

Existing file './data/test_preprocessed.csv' has been deleted.
Test dataFrame has been saved to './data/test_preprocessed.csv'.


### Handling nan values in test_file

In [20]:
from util.DataPreprocess import HandlingMissingValuesTest2
file_name = './data/test_preprocessed.csv'
columns_to_keep = [
    'model',
    'mileage',
    'low mileage car',
    'manufactured',
    'reg_year',
    'type_of_vehicle',
    'dereg_value',
    'depreciation',
    'power',
    'coe',
    'arf',
    'omv',
    'engine_cap',
    'road_tax',
    'almost new car',
    'coe car',
    'parf car',
]

# handling method 1
df_test = HandlingMissingValuesTest2(df_aug, df_test)
# Check for NaN values in the specified columns
nan_columns = df_test[columns_to_keep].isnull().sum()
nan_columns = nan_columns[nan_columns > 0]  # Keep only columns with NaN values

print("Columns containing NaN values and their counts:")
print(nan_columns)


NaN values after handling:  98
Columns containing NaN values and their counts:
model    98
dtype: int64


### Check if train data has all models in test data

In [21]:
models_in_df = set(df['model'].unique())
models_in_df_test = set(df_test['model'].unique())

if models_in_df_test.issubset(models_in_df):
    print("df includes all models in df_test")
else:
    missing_models = models_in_df_test - models_in_df
    print("df does not include", missing_models)

df does not include {nan}


### Mining code here

In [22]:
from util.DataMining import split_dataframe, split_dataframe_flex
from util.DataMining import (
    RandomForestMining,
    RandomForestMiningByModel,
    GradientBoostingMining,
    LinearRegressionMining,
    LinearRegressionMiningByModel,
    CombinedDataMiningRandomForestAndLinearRegression
)

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [23]:
# run_times, rmse_sum = 5, 0
# for i in tqdm(range(run_times), desc='Running Random Forest'):
#     target_col = 'price'
#     x_train, x_test, y_train, y_test = split_dataframe(df, target_col)
#     rmse_sum += RandomForestMining(x_train, x_test, y_train, y_test)
# print('Average RMSE:', round(rmse_sum / run_times))

In [30]:
# from sklearn.model_selection import KFold
# from tqdm import tqdm
# 
# # 定义 K 值
# k = 5
# kf = KFold(n_splits=k, shuffle=True, random_state=42)
# rmse_sum = 0
# for i in tqdm(kf.split(df_aug), desc='Running Random Forest'):
#     train_drop_cols = ['price']
#     test_cols = ['price', 'model']
#     x_train, x_test, y_train, y_test = split_dataframe_flex(df_aug, train_drop_cols, test_cols)
#     rmse_sum += RandomForestMiningByModel(x_train, x_test, y_train, y_test)
# print('Average RMSE:', round(rmse_sum / k))

Running Random Forest: 1it [01:49, 109.50s/it]

Data saved to results.csv
Running not in develop mode
RMSE on test data: 15245.877340628189


Running Random Forest: 2it [03:38, 109.24s/it]

Data saved to results.csv
Running not in develop mode
RMSE on test data: 14990.719666651139


Running Random Forest: 3it [05:27, 109.18s/it]

Data saved to results.csv
Running not in develop mode
RMSE on test data: 15058.312534627223


Running Random Forest: 4it [07:16, 109.00s/it]

Data saved to results.csv
Running not in develop mode
RMSE on test data: 13102.0849458958


Running Random Forest: 5it [09:06, 109.23s/it]

Data saved to results.csv
Running not in develop mode
RMSE on test data: 18484.529850129482
Average RMSE: 15376


In [24]:
run_times, rmse_sum = 1, 0
for i in tqdm(range(run_times), desc='Running Random Forest'):
    train_drop_cols = ['price']
    test_cols = ['price', 'model']
    x_train, x_test, y_train, y_test = split_dataframe_flex(df_aug, train_drop_cols, test_cols)
    rmse_sum += RandomForestMiningByModel(x_train, x_test, y_train, y_test)
print('Average RMSE:', round(rmse_sum / run_times))

Running Random Forest: 100%|██████████| 1/1 [02:04<00:00, 124.13s/it]

Data saved to results.csv
Running not in develop mode
RMSE on test data: 14583.364491397651
Average RMSE: 14583


In [25]:
# x_train, y_train = df.drop(columns=['price']), df['price']
# x_test = df_test[x_train.columns]

# res = RandomForestMining(x_train, x_test, y_train, dev=True)
# res.to_csv('./data/res.csv', index=False)

### This cell do prediction model by model

In [26]:
x_train, y_train = df_aug.drop(columns=['price']), df_aug[['price', 'model']]
x_test = df_test[x_train.columns].dropna(subset=['model'])

res_model = RandomForestMiningByModel(x_train, x_test, y_train, dev=True)
print(res_model.head())

0     19581.940
1     33418.030
2    145927.600
3     72150.145
4     27236.805
Name: Predicted, dtype: float64


### This cell do prediction on test data with 'model' attribute missing

In [32]:
x_train, y_train = df_aug.drop(columns=['price', 'model']), df_aug[['price']]
df_test_unmapped = df_test[df_test['model'].isna()]
x_test = df_test_unmapped[x_train.columns]

res_nomodel = RandomForestMining(x_train, x_test, y_train, dev=True)
print(res_nomodel.head())

C:\Users\17768\anaconda3\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


         Predicted
21    59501.990428
195  295143.191892
212  179589.133772
402   19316.375595
412  105233.445044


In [33]:
print(len(res_model))
print(len(res_nomodel))
res = pd.concat([res_model, res_nomodel])
res.to_csv('./data/res_by_model_original.csv')
res.reset_index(inplace=True)
res.rename(columns={'index': 'Id'}, inplace=True)
res_sorted = res.sort_values(by='Id')

# Fill NaN values in the second column (index 1) with values from the third column (index 2)
res_sorted.iloc[:, 1] = res_sorted.iloc[:, 1].fillna(res_sorted.iloc[:, 2])

# Drop the third column (index 2)
res_sorted = res_sorted.drop(res_sorted.columns[2], axis=1)



# Modify the name of the second column (index 1)
res_sorted.rename(columns={res_sorted.columns[1]: 'Predicted'}, inplace=True)
res_sorted.to_csv('./data/res_by_model2.csv', index=False)
# Display the result
print(res_sorted.head())

9902
98
   Id   Predicted
0   0   19581.940
1   1   33418.030
2   2  145927.600
3   3   72150.145
4   4   27236.805
